In [41]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [43]:
tf.keras.backend.clear_session()

In [44]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [45]:
data = pd.read_csv('2003_mo_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-1,0.254227,8.293882,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,2.290151,887.301063,1472.275657,22.112973,15.990623,71.130346,2.506073,0.826971,1.184017e+08,3.570616e+06,43.639430,1.092465e+08,156.625720,50.047
1,2003-2,0.252655,8.287887,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,2.170793,887.625500,1467.038600,21.790128,15.663933,70.741484,2.494967,0.827063,1.185511e+08,3.573184e+06,43.651300,1.093025e+08,157.678528,48.110
2,2003-3,0.250972,8.281892,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,1.843974,887.995527,1443.186660,21.382001,14.859800,69.073150,2.482659,0.827155,1.187006e+08,3.575752e+06,43.663170,1.093585e+08,159.190268,49.006
3,2003-4,0.249179,8.275896,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,1.842883,888.753058,1484.414910,20.864040,13.463843,65.515153,2.541473,0.827247,1.188500e+08,3.578320e+06,43.675041,1.094145e+08,160.688376,48.445
4,2003-5,0.247070,8.269901,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,1.498497,889.051690,1487.564625,20.593129,12.206035,61.816227,2.562082,0.827340,1.189994e+08,3.580889e+06,43.686911,1.094705e+08,162.847410,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2002-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.285520,888.063858,1554.000510,22.107259,11.557766,54.920321,2.663202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,2002-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.459765,887.482239,1504.599147,22.342946,12.632697,58.260143,2.597288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,2002-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.404292,887.089138,1477.730987,22.495195,13.705746,61.921603,2.551755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,2002-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.375765,887.036669,1469.367571,22.206488,14.859807,66.774435,2.510615,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,0.277737,-0.119544,-0.280381,1.179433,1.185485,0.812336,0.534606,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.091026,0.070157,-0.284744,0.627546,1.008225,0.752759,0.473548,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.100747,0.286513,-0.304616,-0.070129,0.571905,0.497158,0.405876,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.104117,0.729447,-0.270268,-0.955559,-0.185537,-0.047954,0.729237,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-2.168114,0.904059,-0.267644,-1.418668,-0.868019,-0.614656,0.842545,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.057720,0.143581,-0.279406,0.343000,-0.780687,-0.889009,-1.198646,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.942094,-0.181176,-0.289383,0.669601,-0.270236,-0.435587,-1.310058,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.898651,-0.221957,-0.286952,0.821771,0.365487,0.113870,-1.337385,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.302643,-0.280123,-0.278081,0.770652,0.720094,0.515085,-1.438791,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [47]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
259       NaN
260       NaN
261       NaN
262       NaN
263       NaN
Name: Distrito Federal - Consumo de Cimento (t), Length: 264, dtype: float64

In [48]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,0.277737,-0.119544,-0.280381,1.179433,1.185485,0.812336,0.534606,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.091026,0.070157,-0.284744,0.627546,1.008225,0.752759,0.473548,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.100747,0.286513,-0.304616,-0.070129,0.571905,0.497158,0.405876,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.104117,0.729447,-0.270268,-0.955559,-0.185537,-0.047954,0.729237,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-2.168114,0.904059,-0.267644,-1.418668,-0.868019,-0.614656,0.842545,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.937260,1.309227,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,-0.839849,0.040510,-0.187966,2.074338,0.723376,0.038677,-1.173917,1.616226,1.235244,-1.506889,-0.747477,1.174551,0.131039
158,0.925760,1.302282,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,-1.200099,0.377452,-0.223321,1.301918,0.279582,-0.224239,-0.988546,1.563561,1.236423,-1.511843,-0.763990,1.168128,0.077457
159,0.912813,1.295337,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,-1.560955,0.645732,-0.210251,0.688334,-0.489394,-0.857990,-0.768463,1.510895,1.237602,-1.516798,-0.780503,1.161706,0.032090
160,0.906620,1.288392,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,-1.699882,0.898495,-0.222246,0.195905,-1.070006,-1.293797,-0.651774,1.458230,1.238780,-1.521752,-0.797016,1.155283,-0.032489


In [49]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
157    35.599
158    49.981
159    39.881
160    48.598
161    49.942
Name: Distrito Federal - Consumo de Cimento (t), Length: 162, dtype: float64

In [50]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [51]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [52]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
126,0.746963,0.952687,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,-0.613104,0.329520,-0.238450,-0.632531,-1.103927,-1.014495,0.824826,0.686090,0.806860,-0.638912,-0.325093,0.894339,1.712291
127,0.759387,0.973455,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,-0.425963,-0.036556,-0.240084,-0.084677,-0.828398,-0.867761,0.901915,0.670518,0.828599,-0.679356,-0.322764,0.917121,1.675940
128,0.775249,0.994223,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,-0.389430,-0.459764,-0.286784,0.137120,-0.086861,-0.127310,0.768945,0.654947,0.850339,-0.719800,-0.320434,0.939903,1.626852
129,0.795874,1.014990,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,0.470197,-0.665622,-0.274696,0.214746,0.352792,0.309173,0.335289,0.639375,0.872078,-0.760244,-0.318105,0.962685,1.569267
130,0.815598,1.035758,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.981576,-0.862440,-0.270397,0.312303,0.663964,0.600351,0.232565,0.623804,0.893818,-0.800689,-0.315775,0.985467,1.514446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.057720,0.143581,-0.279406,0.343000,-0.780687,-0.889009,-1.198646,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.942094,-0.181176,-0.289383,0.669601,-0.270236,-0.435587,-1.310058,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.898651,-0.221957,-0.286952,0.821771,0.365487,0.113870,-1.337385,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.302643,-0.280123,-0.278081,0.770652,0.720094,0.515085,-1.438791,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [53]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [54]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [55]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [56]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_val_loss = min(history.history['val_loss'][500:])
        current_loss = history.history['loss'][history.history['val_loss'].index(current_val_loss)]
        decision_value = (current_val_loss*current_val_loss)*current_loss 
        print(f"val_loss: {current_val_loss}; loss={current_loss} => decision_value={decision_value}")

        if decision_value < best_loss:
            best_loss = decision_value
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [57]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2894262223, 1230071131, 2482132878, 3335515795, 2236920077, 2494596445, 600642556, 564892824, 2886217333, 3152868891, 3022673020, 1741197285, 1620677528, 1848837477, 1256550277, 2122047413, 3236138532, 672870569, 1822338568, 1481940920]


Step: 0 ___________________________________________
val_loss: 35.434906005859375; loss=17.533445358276367 => decision_value=22015.564944726106
winner_seed: 2894262223


Step: 1 ___________________________________________
val_loss: 29.712141036987305; loss=10.62138557434082 => decision_value=9376.67947223909
winner_seed: 1230071131


Step: 2 ___________________________________________
val_loss: 37.176246643066406; loss=11.275373458862305 => decision_value=15583.39276813299


Step: 3 ___________________________________________
val_loss: 30.54330062866211; loss=12.12997817993164 => decision_value=11315.974321448357


Step: 4 ___________________________________________


2023-10-20 13:55:03.280754: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 31.394622802734375; loss=15.964066505432129 => decision_value=15734.54059978187


Step: 5 ___________________________________________
val_loss: 30.91696548461914; loss=11.861695289611816 => decision_value=11338.105289074114


Step: 6 ___________________________________________
val_loss: 25.62334442138672; loss=14.159295082092285 => decision_value=9296.36701748579
winner_seed: 600642556


Step: 7 ___________________________________________
val_loss: 33.48903274536133; loss=16.87221336364746 => decision_value=18922.445672115995


Step: 8 ___________________________________________
val_loss: 36.33527755737305; loss=11.658345222473145 => decision_value=15391.958203704487


Step: 9 ___________________________________________
val_loss: 32.24418258666992; loss=14.30485725402832 => decision_value=14872.578568137867


Step: 10 ___________________________________________
val_loss: 37.7003059387207; loss=18.38932991027832 => decision_value=26136.994910908965


Step: 11 _________________

In [58]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=3105829098)

Epoch 1/10000
4/4 [==============================] - 2s 147ms/step - loss: 5888.3276 - val_loss: 504.5234
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 423.7693 - val_loss: 882.9650
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 557.6664 - val_loss: 504.8465
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 432.9029 - val_loss: 420.9022
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 391.1613 - val_loss: 390.6550
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 544.7264 - val_loss: 960.5371
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 508.3216 - val_loss: 330.4065
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 226.4603 - val_loss: 245.1810
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 204.8096 - val_loss: 397.3036
Epoch 10/10000
4/4 [==============================] - 0s 10ms/st

4/4 [==============================] - 0s 10ms/step - loss: 39.9662 - val_loss: 65.7232
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 36.4502 - val_loss: 88.6972
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 42.4899 - val_loss: 146.2623
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 82.3382 - val_loss: 87.7652
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.8039 - val_loss: 58.9492
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 25.1617 - val_loss: 68.7839
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.1903 - val_loss: 58.4867
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 35.8092 - val_loss: 63.7490
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 32.2840 - val_loss: 59.8115
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss: 36.2152 - val_

4/4 [==============================] - 0s 9ms/step - loss: 20.7184 - val_loss: 42.5899
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 23.2536 - val_loss: 41.6776
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.8283 - val_loss: 49.0341
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.7843 - val_loss: 50.4282
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 26.0182 - val_loss: 46.8607
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 25.2529 - val_loss: 56.8152
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.4069 - val_loss: 41.5341
Epoch 166/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.1765 - val_loss: 50.7319
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 27.8138 - val_loss: 50.5026
Epoch 168/10000
4/4 [==============================] - 0s 10ms/step - loss: 28.268

4/4 [==============================] - 0s 10ms/step - loss: 20.9541 - val_loss: 50.6416
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 35.1969 - val_loss: 67.5699
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.5254 - val_loss: 44.7599
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 22.8280 - val_loss: 43.6471
Epoch 242/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.9881 - val_loss: 45.9296
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.5042 - val_loss: 35.3468
Epoch 244/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.2637 - val_loss: 53.0754
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 27.5400 - val_loss: 43.6826
Epoch 246/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.4472 - val_loss: 42.9381
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.6

4/4 [==============================] - 0s 9ms/step - loss: 15.4119 - val_loss: 42.5237
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.4987 - val_loss: 36.4497
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.0242 - val_loss: 44.4174
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.5119 - val_loss: 43.8379
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.5175 - val_loss: 55.5637
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.8919 - val_loss: 48.0410
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.2868 - val_loss: 41.2904
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.0787 - val_loss: 42.3372
Epoch 325/10000
4/4 [==============================] - 0s 10ms/step - loss: 20.1224 - val_loss: 52.3191
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 25.2498 - va

...

Epoch 791/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.1315 - val_loss: 33.5471
Epoch 792/10000
4/4 [==============================] - 0s 9ms/step - loss: 22.9337 - val_loss: 30.2826
Epoch 793/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.6296 - val_loss: 33.5596
Epoch 794/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.5506 - val_loss: 35.1772
Epoch 795/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.8132 - val_loss: 36.6804
Epoch 796/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5701 - val_loss: 38.1504
Epoch 797/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.0929 - val_loss: 35.0390
Epoch 798/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.8362 - val_loss: 34.1559
Epoch 799/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.3007 - val_loss: 28.7604
Epoch 800/10000
4/4 [==============================] - 0s 10ms/step

Epoch 870/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.6427 - val_loss: 34.4787
Epoch 871/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.4501 - val_loss: 39.3975
Epoch 872/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.3739 - val_loss: 46.2083
Epoch 873/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.1782 - val_loss: 35.7886
Epoch 874/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.9451 - val_loss: 35.5601
Epoch 875/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.1694 - val_loss: 37.7369
Epoch 876/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.3840 - val_loss: 31.9510
Epoch 877/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.3687 - val_loss: 40.3786
Epoch 878/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.4378 - val_loss: 39.0815
Epoch 879/10000
4/4 [==============================] - 0s 11ms/ste

Epoch 949/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.4922 - val_loss: 34.2571
Epoch 950/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1660 - val_loss: 30.3439
Epoch 951/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.8967 - val_loss: 31.1517
Epoch 952/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.3733 - val_loss: 45.6814
Epoch 953/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1494 - val_loss: 29.4040
Epoch 954/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.5466 - val_loss: 38.2669
Epoch 955/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5567 - val_loss: 37.3629
Epoch 956/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.6048 - val_loss: 39.3158
Epoch 957/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5973 - val_loss: 32.6372
Epoch 958/10000
4/4 [==============================] - 0s 10ms/s

4/4 [==============================] - 0s 9ms/step - loss: 14.7938 - val_loss: 38.7917
Epoch 1028/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.8264 - val_loss: 37.6561
Epoch 1029/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.5679 - val_loss: 35.5171
Epoch 1030/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.7422 - val_loss: 36.4083
Epoch 1031/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4165 - val_loss: 34.4503
Epoch 1032/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.0411 - val_loss: 37.1156
Epoch 1033/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.3296 - val_loss: 33.6760
Epoch 1034/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.0795 - val_loss: 29.4677
Epoch 1035/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.7729 - val_loss: 29.3193
Epoch 1036/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.274

Epoch 1106/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.1625 - val_loss: 37.7018
Epoch 1107/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.0191 - val_loss: 32.5835
Epoch 1108/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.7733 - val_loss: 36.1462
Epoch 1109/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.3403 - val_loss: 34.7032
Epoch 1110/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.7751 - val_loss: 36.0293
Epoch 1111/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.9756 - val_loss: 34.9997
Epoch 1112/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.7737 - val_loss: 35.1628
Epoch 1113/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.4773 - val_loss: 35.1224
Epoch 1114/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.8391 - val_loss: 36.8186
Epoch 1115/10000
4/4 [==============================] - 0s

In [59]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [60]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,47.451515,47.844711,47.599411,46.545891,45.436623,44.47319,44.036385,44.003807,44.057716,44.518478,46.007637,47.237801,47.891228,48.308281,48.693069,48.783073,47.935478,45.978165,44.921684,44.382416,44.20993,45.034363,46.442513,47.487762,48.125183,48.380981,48.195152,47.784645,46.879585,45.753296
Target,53.235,59.29,54.875,57.52,49.573,40.698,47.195,43.527,45.488,44.96,36.721,54.463,51.462,58.854,53.94,51.514,42.635,40.371,48.99,44.726,44.491,43.901,55.019,50.942,63.992,64.127,60.822,65.182,51.282,43.554
Error,5.783485,11.44529,7.275589,10.97411,4.136379,3.775188,3.158615,0.476807,1.430283,0.441521,9.286636,7.225201,3.570774,10.545719,5.246929,2.730927,5.30048,5.607166,4.068317,0.343586,0.281071,1.133362,8.576488,3.454239,15.866817,15.746017,12.626846,17.397354,4.402416,2.199295


In [61]:
display(mae)
display(mape)

6.1502304

0.11396409

In [62]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [63]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined